In [1]:
import ee

ee.Initialize()

In [2]:
from pprint import pprint
data = ee.FeatureCollection("projects/ee-nwrc-geomatics/assets/cnwi/sampled_features_122")

train = data.filter(ee.Filter.eq('type', 1))
test = data.filter(ee.Filter.eq('type', 2))



# get the predictors
predictors = data.first().propertyNames().removeAll(['type','region_id', 'system:index', 'system:time_start', 'system:time_end', 'system:footprint'])
pprint(predictors.getInfo())

['HH',
 'B8A_1',
 'B8A_2',
 'B8A_3',
 'brightness_1',
 'brightness_2',
 'HV',
 'SAVI_2',
 'SAVI_3',
 'B11_2',
 'B11_1',
 'SAVI_1',
 'B11_3',
 'VH',
 'HH_HV',
 'B7_2',
 'B7_3',
 'B5_2',
 'B7_1',
 'B5_3',
 'B3_2',
 'B5_1',
 'B3_3',
 'B3_1',
 'VV',
 'wetness_2',
 'wetness',
 'NDVI_1',
 'NDVI_2',
 'wetness_1',
 'class_name',
 'NDVI_3',
 'greenness',
 'B12_1',
 'B12_3',
 'B12_2',
 'brightness',
 'B8_3',
 'VV_VH',
 'B8_1',
 'B6_3',
 'B8_2',
 'B6_1',
 'B4_3',
 'B6_2',
 'B4_1',
 'B2_3',
 'greenness_1',
 'B4_2',
 'greenness_2',
 'B2_1',
 'B2_2']


In [3]:
from cnwi.modeling import SmileRandomForest

rfm = SmileRandomForest(numberOfTrees=100)
rfm.fit(train, inputProperties=predictors, classProperty='class_name')

pprint(rfm.model.getInfo())


{'classProperty': 'class_name',
 'classifier': {'classifier': {'numberOfTrees': 100,
                               'type': 'Classifier.smileRandomForest'},
                'mode': 'CLASSIFICATION',
                'type': 'Classifier.setOutputMode'},
 'features': {'columns': {'B11_1': 'Integer',
                          'B11_2': 'Integer',
                          'B11_3': 'Integer',
                          'B12_1': 'Integer',
                          'B12_2': 'Integer',
                          'B12_3': 'Integer',
                          'B2_1': 'Integer',
                          'B2_2': 'Integer',
                          'B2_3': 'Integer',
                          'B3_1': 'Integer',
                          'B3_2': 'Integer',
                          'B3_3': 'Integer',
                          'B4_1': 'Integer',
                          'B4_2': 'Integer',
                          'B4_3': 'Integer',
                          'B5_1': 'Integer',
                      

In [4]:
# do assessment
from cnwi.metrics import Metrics
test_predictions = rfm.predict(test)
metrics = Metrics(test_predictions, 'class_name', class_order = data.aggregate_array('class_name').distinct())
metrics = metrics.create_metrics()



In [5]:
import time
task = ee.batch.Export.table.toDrive(
    collection = metrics,
    description = 'test_predictions',
    folder = 'cnwi_122_test_metrics',
    fileFormat = 'GeoJSON'
)

task.start()

while task.status()['state'] in ['READY', 'RUNNING']:
    print(task.status())
    print("------------------")
    time.sleep(5)
task.status()

{'state': 'READY', 'description': 'test_predictions', 'creation_timestamp_ms': 1705071171735, 'update_timestamp_ms': 1705071171735, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'CHUHETURJYRZRD7JNJFMBJ5J', 'name': 'projects/earthengine-legacy/operations/CHUHETURJYRZRD7JNJFMBJ5J'}
------------------


In [8]:
from cnwi.metrics import build_metrics_table
fname = "./test_predictions.geojson"
table = build_metrics_table(fname)


In [9]:
table

,1,2,3,4,5,6,7,Producers
1,4.00,0.0,1.0,2.0,1.0,0.0,0.0,50.00
2,9.00,174.0,0.0,5.0,1.0,0.0,0.0,92.06
3,2.00,0.0,14.0,0.0,4.0,0.0,7.0,51.85
4,0.00,0.0,0.0,6.0,0.0,0.0,0.0,100.00
5,1.00,0.0,0.0,0.0,1.0,2.0,0.0,25.00
6,0.00,0.0,0.0,0.0,1.0,3.0,0.0,75.00
7,0.00,0.0,2.0,1.0,0.0,0.0,13.0,81.25
Consumers,25.00,100.0,82.0,43.0,12.0,60.0,65.0,NaN
Overall Accuracy,84.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN
